# Import Library

In [2]:
import os
import fnmatch as fn
import numpy as np
import pandas as pd
#!pip install dbfread

import dbfread

#!pip install pandas_profiling
import pandas_profiling as pp
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from matplotlib import font_manager, rc
%matplotlib inline
#%matplotlib notebook

import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
from math import sin,cos,sqrt,atan2,radians 
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import shapiro
from scipy.stats import skew

import requests
import logging
import time
import googlemaps
import reverse_geocoder
import pprint
import folium
import zipfile
import json
#%matplotlib.inline
import itertools

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import lightgbm as lgb

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

matplotlib.rcParams['axes.unicode_minus'] = False # 마이너스 기호도 표시

# 한글 깨짐 방지 목적 #
font_name = font_manager.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family = font_name)

## 맥
# rc('font', family='/Library/Fonts/AppleGothic.ttf')


# 만약 plotly가 설치 되지 않았을 경우 하단의 코드를 실행할 것 #
#!pip install plotly
import plotly
#!pip install cufflinks
#import plotly.express as px
# Cufflinks wrapper on plotly
import cufflinks as cf
import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff
from plotly.offline import iplot
from plotly.offline import init_notebook_mode, plot,iplot

init_notebook_mode(connected = True)
cf.go_offline()

import warnings
warnings.filterwarnings(action='ignore') # 경고 메시지를 숨길 때

# Import Data

In [3]:
df = pd.read_csv('가공데이터/merge_유통_유동_환경_소상공인_edit1.csv')
df = df.iloc[:,1:]
df['date'] = pd.to_datetime(df['date'],format = '%Y-%m-%d')

df_revised = pd.read_csv('가공데이터/merge_유통_유동_환경_소상공인_edit2.csv')
df_revised = df_revised.iloc[:,1:]
df_revised['date'] = pd.to_datetime(df_revised['date'],format = '%Y-%m-%d')

gs_retail = pd.read_csv('가공데이터/GS편의점_종로노원구.csv')
gs_retail = gs_retail.iloc[:,1:]

tp_revised = pd.read_csv('가공데이터/유동_edit1.csv')
tp_revised = tp_revised.iloc[:,1:]
tp_revised = tp_revised.drop(['STD_YM','STD_YMD','HDONG_CD'],axis=1)
tp_revised['date'] = pd.to_datetime(tp_revised['date'],format='%Y-%m-%d')

In [9]:
df.columns

Index(['date', 'year', 'quarter', 'month', 'weekday', 'day', 'district',
       'p10_most_freq_day', 'p10_mean_day', 'p10_25per', 'p10_median_day',
       'p10_75per', 'p25_most_freq_day', 'p25_mean_day', 'p25_25per',
       'p25_median_day', 'p25_75per', 'humi_mean_day', 'humi_median_day',
       'humi_25per', 'humi_75per', 'temp_mean_day', 'temp_median_day',
       'temp_25per', 'temp_75per', 'noise_mean_day', 'noise_median_day',
       'dis_index_most_freq2', 'MAN_0004', 'MAN_0509', 'MAN_1014', 'MAN_1519',
       'MAN_2024', 'MAN_2529', 'MAN_3034', 'MAN_3539', 'MAN_4044', 'MAN_4549',
       'MAN_5054', 'MAN_5559', 'MAN_6064', 'MAN_6569', 'MAN_70U', 'WMAN_0004',
       'WMAN_0509', 'WMAN_1014', 'WMAN_1519', 'WMAN_2024', 'WMAN_2529',
       'WMAN_3034', 'WMAN_3539', 'WMAN_4044', 'WMAN_4549', 'WMAN_5054',
       'WMAN_5559', 'WMAN_6064', 'WMAN_6569', 'WMAN_70U', 'SGNG_NM', 'values',
       'LENGTH', 'AREA', '10_P', '20_P', '30_P', '40_P', '50_P', '60_P',
       '70_P', '80_P', 'AMT_IND

In [4]:
shape_visual = gpd.read_file('Rawdata/유동인구데이터/행정동경계파일/종로_노원_행정동.shp',
                             encoding = 'euckr')

ad = dbfread.DBF('Rawdata/유동인구데이터/행정동경계파일/종로_노원_행정동.dbf')
ad = pd.DataFrame(iter(ad))

ad['X_COORD'] = ad['X_COORD'].apply(float)
ad['Y_COORD'] = ad['Y_COORD'].apply(float)

g_info = [Point(i) for i in zip(ad['X_COORD'],ad['Y_COORD'])]

geo_ad = gpd.GeoDataFrame(ad,
                          crs = {'init' : 'epsg:4326'},
                          geometry = g_info
                         )
geo_ad.head()

,STD_YYYY,HDONG_CD,HDONG_NM,SIDO_CD,SIDO_NM,SGNG_CD,SGNG_NM,AREA,LENGTH,X_COORD,Y_COORD,geometry
0,2019,1111051500,청운효자동,11,서울특별시,11110,종로구,2573175.62,7975.05,126.972694,37.581306,POINT (126.972694 37.581306)
1,2019,1111053000,사직동,11,서울특별시,11110,종로구,1128161.02,5498.40,126.970919,37.573408,POINT (126.9709195 37.573408)
2,2019,1111054000,삼청동,11,서울특별시,11110,종로구,1478468.25,7377.76,126.984028,37.582083,POINT (126.984028 37.582083)
3,2019,1111055000,부암동,11,서울특별시,11110,종로구,2276588.99,8912.71,126.966500,37.589917,POINT (126.9665 37.589917)
4,2019,1111056000,평창동,11,서울특별시,11110,종로구,8951336.18,13312.43,126.968972,37.602583,POINT (126.968972 37.602583)


# Definition of Problem

- 환경 데이터, 유동 데이터, 그리고 T시점 이전의 유통지수를 사용하여 동 및 카테고리별 유통 판매 지수를 예측한다.
- 이 경우, 해당 문제는 Sequence Prediction에 해당된다.

## sequence Prediction Classification

- Sequence Prediction은 크게 'One-to-One Model', 'One-to-Many Model', 'Many-to-One model', 'Many-to-Many model'이 있다.
- 이 네 가지 유형들 중에서 현재의 문제는 Multiple Input values를 모델이 받아서 single Output을 도출하는 구조이므로 'Many-To-one' model에 속한다.
- 우리가 해결할 문제는 Multivariate & Multi-Step Time Series Forecasting Problem임이 자명하다.

# Data Preprocessing 

In [ ]:
df.head()

## 구체적인 전처리

### 먼저 전처리의 지역 대상은 동으로 한정한다. 왜냐하면 지수 및 모델의 효과가 동을 기준으로 존재하기 때문이다. 

### 다음에 불필요한 변수들을 제거한다. 

### 만약 변수가 정수 또는 부동소수라면 Normalization method 또는 Standardization method를 거친다.

#### Normalization은 범위를 0과 1 사이에 모든 값들을 Rescaling하는 작업이다. Normalization은 관측치의 최대값과 최솟값을 정확하게 추정할 수 있어야 한다.

#### Standardization은 관측치의 평균을 0으로 그리고 분산을 1로 만드는 Rescaling method이다. Standardization은 관측치가 가우시안 분포를 따른다고 전제한다. 다시 말해서, 평균과 표준편차를 정확하게 추정하고 있어야함을 의미한다.

### 범주형 변수 Handling

#### 범주형 변수의 범주는 정수형이 아니다. 가능한 범주가 K개가 있다면 1~k까지의 정수를 부여하는 것이 간단학 쉬어 보인다. 하지만 그 결과 값은 서로에 대해 순서를 매길 수 있기 때문에 범주로서 허용되지 않는다. 몇 가지 대안이 있다.

##### One-hot encoding
    - 모든 비트의 합은 1 --> 선형 종속성 --> 선형 모델의 고유성 조건 깨짐
    
##### Dummy Coding
    - 표현식에서 k-1개만을 사용해서 자유도를 k에서 k-1로 줄인다.

In [ ]:
df.columns

### Testset과 Trainset을 분리하여 첫번째 전처리 작업을 진행한다.

- 시계열 데이터이므로 CV(Cross Validation)은 적용할 수 없다.

In [ ]:
def pre_processing1(data):
    
    df_copy = data.copy()
    
    y = []
    y2 = []
    
    for i in np.unique(df_copy['district']):
        df2 = df_copy[df_copy['district'] == i]

        df2.index = df2['date']
        df2.drop(['date','quarter','day','LENGTH','AREA','SGNG_NM','cri',
                  '10_P', '20_P', '30_P', '40_P', '50_P', '60_P','district',
                  '70_P', '80_P'],
                 axis=1,
                 inplace=True)
        
        target = ['values', 'AMT_IND_10', 'AMT_IND_20', 'AMT_IND_30',
        'AMT_IND_40', 'AMT_IND_50', 'AMT_IND_60', 'AMT_IND_70', 'AMT_IND_80']
        
        x = df2.loc[:,~(df2.columns.isin(target))]
        x_t = df2.loc[:,(df2.columns.isin(target))]
        
        df2['year'] = df2['year'].astype('str')
        df2['month'] = df2['month'].astype('str')

        for i in x.columns:
            if ((x[i].dtypes == 'int')|(x[i].dtypes == 'float')):
                if ((skew(x[i]) > -2) & (skew(x[i]) < 2)):
                    x[i] = x[i]
                else:
                    x[i] = np.log(x[i])
                    if ((skew(x[i]) > -2) & (skew(x[i]) < 2)):
                        x[i] = np.exp(x[i])
                        x[i] = np.sqrt(x[i])
            else:
                continue


        x2 = x.loc[:,(x.dtypes == 'int')|(x.dtypes == 'float')]
        normalized_x2 = pd.DataFrame((x2 - x2.mean())/x2.std())
        #normalized_x2.rename(x2.columns,inplace=True)
        #normalized_x2.index = df2.index

        cate = x.loc[:,x.dtypes == 'object'].columns
        df3 = pd.get_dummies(x.loc[:,cate])
        df2 = pd.concat([normalized_x2,df3],axis=1)
        df2 = df2.loc[:,~(df2.columns.isin(cate))]
        df_final = pd.concat([df2,x_t],axis=1)
        y.append(df_final)
        #y2= pd.concat(y[0:len(y)])
    return y

In [ ]:
y = pre_processing1(df)

In [ ]:
print(np.unique(df['district']))
print(len(np.unique(df['district'])))
print(len(y))
print(len(y2))

## Convert Sequence into Supervised Problem format

In [ ]:
target = ['values','AMT_IND_10', 'AMT_IND_20', 'AMT_IND_30',
        'AMT_IND_40', 'AMT_IND_50', 'AMT_IND_60', 'AMT_IND_70', 'AMT_IND_80']
x_var = y[0].loc[:,~(y[0].columns.isin(target))].columns

In [ ]:
def pre_processing2(data,input_steps,output_steps,var):
    x = []
    y = []
    
    data = data.reset_index()
    data = data.dropna(axis='columns')
       
    for i in range(len(data)):
        end_ix = i + input_steps
        output_ix = end_ix + output_steps -1
        
        if (output_ix > len(data)-6):
            break
            
        x_seq = data.loc[i:end_ix-1,:]
        y_seq = data.loc[end_ix:output_ix,var]
        x_seq = x_seq.iloc[:,2:]
     #   y_seq = y_seq.iloc[:,0:]
        
        x_seq = x_seq.values
        y_seq= y_seq.values
        
        x_seq = x_seq.reshape(x_seq.shape)
#        y_seq = y_seq.values
        y_seq = y_seq.reshape(y_seq.shape)
        x.append(x_seq)
        y.append(y_seq)
    return np.array(x),np.array(y)

In [ ]:
input_steps = 7
output_steps = 7
var = target[0]
trainx,trainy = pre_processing2(y[0],input_steps,output_steps,var)
trainy = np.array([ab.reshape(1,7).tolist() for ab in trainy]).reshape((-1,7))
print(trainx.shape)
print(trainy.shape)

In [ ]:
testx,testy = pre_processing2(y2[0],input_steps,output_steps)
testx.shape

# Modeling


## LSTM State Management

### The Efficiency of learning : 업데이트 전에 얼마만큼의 샘플들을 학습하여야 하는가?

### LSTM 입력층은 3차원이어야 한다.
- 3차원 구성 : [samples, time steps, features]

### The Speed of Learning : 가중치들은 얼마만큼의 업데이트 주기를 가지는가?

### LSTM 내부 상태가 얼마나 자주 reset되어야 하는가?

## Stacked LSTM

In [ ]:
def predict_lstm(var):
    
    plate1,plate2 = [], []
    
    for i in range(len(y)):
        input_steps = 7
        output_steps = 7
        
        trainx,trainy = pre_processing2(y[i],input_steps,output_steps,var)
      #  testx,testy = pre_processing2(y2[i],input_steps,output_steps)

        trainy = np.array([ab.reshape(1,7).tolist() for ab in trainy]).reshape((-1,7))
      #  testy = np.array([ab.reshape(1,7).tolist() for ab in testy]).reshape((-1,7))

        n_features = trainx.shape[2]

        # Define model
        model = Sequential()
        model.add(LSTM(150,
                        activation='relu',
                       return_sequences = True,
                        input_shape=(input_steps,n_features)))
        model.add(LSTM(150,
                        activation='relu'))
        model.add(Dense(output_steps))

        # Compile model
        model.compile(optimizer = 'adam', loss = 'mse',metrics = ['accuracy'])

        # fit model
        fit_lstm = model.fit(trainx,
                             trainy,
                             shuffle = False,
                             epochs=100,
                             verbose=1,
                             validation_split = 0.2)
        
        print(fit_lstm.history['loss'])
        print(fit_lstm.history['acc'])
        print(fit_lstm.history['val_loss'])
        print(fit_lstm.history['val_acc'])

In [ ]:
predict_lstm('values')

In [ ]:
predict_lstm(target[1])

In [ ]:
predict_lstm(target[2])

In [ ]:
predict_lstm(target[3])

In [ ]:
predict_lstm(target[4])

In [ ]:
predict_lstm(target[5])

In [ ]:
predict_lstm(target[6])

In [ ]:
predict_lstm(target[7])

In [ ]:
predict_lstm(target[8])